# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc70475a9a0>
├── 'a' --> tensor([[ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541]])
└── 'x' --> <FastTreeValue 0x7fc70475a9d0>
    └── 'c' --> tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                        [-1.1999,  1.6948,  0.9551, -0.0540],
                        [-0.0651,  0.6188, -2.7386,  0.7798]])

In [4]:
t.a

tensor([[ 1.1698,  1.5184, -0.8098],
        [ 0.0796, -0.5881,  1.0541]])

In [5]:
%timeit t.a

64.3 ns ± 0.0585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc70475a9a0>
├── 'a' --> tensor([[-1.1700, -0.6928, -1.5694],
│                   [-1.2644, -0.7546,  0.1441]])
└── 'x' --> <FastTreeValue 0x7fc70475a9d0>
    └── 'c' --> tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                        [-1.1999,  1.6948,  0.9551, -0.0540],
                        [-0.0651,  0.6188, -2.7386,  0.7798]])

In [7]:
%timeit t.a = new_value

67.4 ns ± 0.0549 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541]]),
    x: Batch(
           c: tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                      [-1.1999,  1.6948,  0.9551, -0.0540],
                      [-0.0651,  0.6188, -2.7386,  0.7798]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1698,  1.5184, -0.8098],
        [ 0.0796, -0.5881,  1.0541]])

In [11]:
%timeit b.a

57.8 ns ± 0.146 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0506, -0.7726,  0.5234],
               [-0.5417, -0.6985,  1.1739]]),
    x: Batch(
           c: tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                      [-1.1999,  1.6948,  0.9551, -0.0540],
                      [-0.0651,  0.6188, -2.7386,  0.7798]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 1.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

816 ns ± 0.195 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 18.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

148 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

147 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc7047540d0>
├── 'a' --> tensor([[[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]],
│           
│                   [[ 1.1698,  1.5184, -0.8098],
│                    [ 0.0796, -0.5881,  1.0541]]])
└── 'x' --> <FastTreeValue 0x7fc65f03e520>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 47.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc7047547f0>
├── 'a' --> tensor([[ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541],
│                   [ 1.1698,  1.5184, -0.8098],
│                   [ 0.0796, -0.5881,  1.0541]])
└── 'x' --> <FastTreeValue 0x7fc704754070>
    └── 'c' --> tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                        [-1.1999,  1.6948,  0.9551, -0.0540],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 72.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 44.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]],
       
               [[ 1.1698,  1.5184, -0.8098],
                [ 0.0796, -0.5881,  1.0541]]]),
    x: Batch(
           c: tensor([[[-0.9837, -0.4014,  1.3998,  0.8342],
                       [-1.1999,  1.6948,  0.9551, -0.0540],
                       [-0.0651,  0.6188, -2.7386,  0.7798]],
         

In [26]:
%timeit Batch.stack(batches)

79.9 µs ± 956 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541],
               [ 1.1698,  1.5184, -0.8098],
               [ 0.0796, -0.5881,  1.0541]]),
    x: Batch(
           c: tensor([[-0.9837, -0.4014,  1.3998,  0.8342],
                      [-1.1999,  1.6948,  0.9551, -0.0540],
                      [-0.0651,  0.6188, -2.7386,  0.7798],
                      [-0.9837, -0.4014,  1.3998,  0.8342],
                      [-1.1999,  

In [28]:
%timeit Batch.cat(batches)

165 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

334 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
